Migrate SQL Server Database to Azure SQL Server VM
================================

# Prerequisites

The SQL Server Instance notebook is a collection of PowerShell scripts integrated together to provide a step-by-step configuration process to do an offline migration of SQL Server Instance on-premises to a SQL Server running in Microsoft Azure.  To execute this notebook there are some resource prerequisites as well as some local machine prerequisites.  To successfully execute this notebook you will need to:
  * Install required PowerShell modules
  * Connect to Azure
  * Have access to the source SQL Instance to migrate from
  * Have access to the target SQL Instance to migrate to

## Install PowerShell Modules 
The scripts in this notebook leverage a few PowerShell modules:

|Module|Description|
|---|---|
|dbatools| Provides high-level CMDLETs that perform SQL management operations.  These CMDLETs have extensive validation tests and configuration options.|
|Az.Accounts| Provides CMDLETs to configure subscription and account settings|
|Az.Resources| Provides CMDLETs to access and provision Microsoft Azure resources|
|Az.Storage| Provides CMDLETs to simplify configuration and access to Azure storage resources|
|GraphicalTools| Provides the cross-platform version of Out-GridView which is used to easily select items from lists|

To execute the scripts these modules should be installed from the PowerShell gallery onto the local machine.  General information on the PowerShell Gallery can be found at https://www.powershellgallery.com/.  If you have not already registered the PowerShell gallery the following code will temporarily configure the PS gallery, install the modules for the current user and then unregister the gallery.  The code installs the required modules for the current user scope.

*Note: this temporary gallery registration is setup as trusted to enable installation of modules without prompting.*




In [1]:
Register-PSRepository -Name PSGalleryUser -SourceLocation https://www.powershellgallery.com/api/v2/ -PublishLocation https://www.powershellgallery.com/api/v2/package/ -ScriptSourceLocation https://www.powershellgallery.com/api/v2/items/psscript/ -ScriptPublishLocation https://www.powershellgallery.com/api/v2/package/ -InstallationPolicy Trusted -PackageManagementProvider NuGet

Install-Module dbatools -Scope CurrentUser -AllowClobber -Repository PSGalleryUser
Install-Module Az.Accounts -Scope CurrentUser -AllowClobber -Repository PSGalleryUser
Install-Module Az.Resources -Scope CurrentUser -AllowClobber -Repository PSGalleryUser
Install-Module Az.Storage -Scope CurrentUser -AllowClobber -Repository PSGalleryUser
Install-Module Microsoft.PowerShell.GraphicalTools -Scope CurrentUser -AllowClobber -Repository PSGalleryUser

Unregister-PSRepository -Name PSGalleryUser

Get-InstalledModule -Name dbatools, Az.Accounts, Az.Resources, Az.Storage, Microsoft.PowerShell.GraphicalTools

## Source SQL Instance
The following code is used to specify the source SQL Server instance.  Data and Server objects will be copied from this server to the target SQL Server instance.  In the following code cell set the following parameters:

*Note: the notebook currently is setup for SQL Authentication.  Future updates will add support for multiple authentication types.*

|Parameter|Description|
|---|---|
|sourceServerName| The name or IP address of the source instance|
|sourceLogin| sql login to connect to source instance with |

*Note: source password should be set in the environment variable SQLMIG_SourcePassword.  This is to avoid persisting the environment variable in the notebook file.*

Edit the code below to specify the above parameters to test connectivity to the source instance.




In [2]:
$sourceServerName = 'sqltools2016-3'
$sourceLogin = 'migtest'

## TEMP - REMOVE BEFORE PUSHING CHANGES
$env:SQLMIG_SourcePassword = 'Yukon900'

## PowerShell Environment 
$sourceLoginPassword = ConvertTo-SecureString $env:SQLMIG_SourcePassword -AsPlaintext -Force
$sourceCredential = New-Object System.Management.Automation.PSCredential ('migtest', $sourceLoginPassword)
$sourceTest = Test-DbaConnection -SqlInstance $sourceServerName -SqlCredential $sourceCredential
$sourceTest
$sourceConnection = Connect-DbaInstance -SqlInstance $sourceServerName -SqlCredential $sourceCredential



ComputerName         : SQLTools2016-3
InstanceName         : MSSQLSERVER
SqlInstance          : sqltools2016-3
SqlVersion           : 13.0.5975
ConnectingAsUser     : migtest
ConnectSuccess       : True
AuthType             : SQL Authentication
AuthScheme           : SQL
TcpPort              : 1433
IPAddress            : 10.116.97.140
NetBiosName          : SQLTools2016-3.redmond.corp.microsoft.com
IsPingable           : True
PSRemotingAccessible : True
DomainName           : redmond.corp.microsoft.com
LocalWindows         : 10.0.18363.0
LocalPowerShell      : 5.1.18362.145
LocalCLR             : 4.0.30319.42000
LocalSMOVersion      : 15.100.0.0
LocalDomainUser      : True
LocalRunAsAdmin      : False
LocalEdition         : Desktop





## Target SQL Instance
The following code is used to specify the target SQL Server instance.  This is the SQL Server instance that Data and Server objects will be copied to.  In the following code cell set the following parameters:

*Note: the notebook currently is setup for SQL Authentication.  Future updates will add support for multiple authentication types.*

|Parameter|Description|
|---|---|
|targetServerName| The name or IP address of the target instance|
|targetLogin| sql login to connect to target instance with |

*Note: the target login password should be set in the environment variable SQLMIG_TargetPassword.  Thisis to avoid persisting the environment variable in the notebook file.*

Edit the code below to specify the above parameters to test connectivity to the target instance.

In [3]:
$targetServerName = 'sqlmig.westus2.cloudapp.azure.com'
$targetLogin = 'cloudsa'

## TEMP - REMOVE BEFORE PUSHING CHANGES
$env:SQLMIG_TargetPassword = 'Yukon900Yukon900'

## PowerShell Environment 
$targetLoginPassword = ConvertTo-SecureString $env:SQLMIG_TargetPassword -AsPlaintext -Force
$targetCredential = New-Object System.Management.Automation.PSCredential ('migtest', $targetLoginPassword)
$targetTest = Test-DbaConnection -SqlInstance $targetServerName -SqlCredential $targetCredential
$targetTest
$targetConnection = Connect-DbaInstance -SqlInstance $targetServerName -SqlCredential $targetCredential



ComputerName         : SQLMIG
InstanceName         : MSSQLSERVER
SqlInstance          : sqlmig.westus2.cloudapp.azure.com
SqlVersion           : 14.0.3257
ConnectingAsUser     : migtest
ConnectSuccess       : True
AuthType             : SQL Authentication
AuthScheme           : SQL
TcpPort              : No such host is known
IPAddress            : 52.183.117.103
NetBiosName          : sqlmig.westus2.cloudapp.azure.com
IsPingable           : False
PSRemotingAccessible : [sqlmig.westus2.cloudapp.azure.com] Connecting to remote server 
                       sqlmig.westus2.cloudapp.azure.com failed with the following error message : WinRM cannot 
                       process the request. The following error occurred while using Kerberos authentication: Cannot 
                       find the computer sqlmig.westus2.cloudapp.azure.com. Verify that the computer exists on the 
                       network and that the name provided is spelled correctly. For more information, see the 


## Login to Microsoft Azure
To configure and provision resources you must log into your Azure account and set the current subscription that is being used for the target SQL Server instance.  The following code will help you connect your account and choose the correct subscription.  When presented with the list of subscriptions, click on the desired subscription and press OK.

In [4]:
#Connect-AzAccount
$migrationSubscription = Get-AzSubscription | Select-Object -Property Name, Id | Out-GridView -PassThru
Set-AzContext -SubscriptionId $migrationSubscription.Id


Name                                     Account             SubscriptionName    Environment         TenantId          
----                                     -------             ----------------    -----------         --------          
ClientToolsInfra_nonExpressRoute (172... kenvh@microsoft.com ClientToolsInfra... AzureCloud          72f988bf-86f1-4...




## Verify No Active Connections

In [24]:
#TODO - filter connected proceesses for user connections


Get-DbaProcess -SqlInstance $SourceServerName -SqlCredential $sourceLogin | 
Select Host, login, Program

## Temporary Storage for Data Movement

Offline data migration attempts to use backup to URL and restore from URL as the mechanism for moving data from the source instance to the target instance.  This code will check existance of the specified storage account and container to use for data migration. If the resources do not exist they will be created.

In [8]:
$resourceGroup = "sqlmig"
$blobStorageAccount = "tempsqlmigstorage"
$containerName = "backups"
$location = "West US 2"

# Storage Account
$storageAccount = Get-AzStorageAccount -ResourceGroupName $resourceGroup -Name $blobStorageAccount
if ($storageAccount -eq $null)
{
    # specified storage account does not yet exist, attempt to create it
    $storageAccount = New-AzStorageAccount -ResourceGroupName $resourceGroup -Name $blobStorageAccount -Location $location -SkuName Standard_LRS -Kind StorageV2
}
$storageAccount

# Container
$storageContext = $storageAccount.Context
$storageContainer = Get-AzStorageContainer -Name $containerName -Context $storageContext
if ($storageContainer -eq $null)
{
    #specified storage container does not yet exist, attempt to create it
    $storageContainer = New-AzStorageContainer -Name $containerName -Context $storageContext -Permission Container
}
$storageContainer

# Provide source instance with SAS token for blob access
$sourceSAS = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "rw" -Context $storageContext).TrimStart('?')
$sourceCred = New-DbaCredential -SqlInstance $sourceConnection -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" -Identity "SHARED ACCESS SIGNATURE" -SecurePassword (ConvertTo-SecureString $sourceSAS -AsPlainText -Force) -Force
$sourceCred

$targetSAS = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "rw" -Context $storageContext).TrimStart('?') # -ResourceType Container,Object
$targetCred = New-DbaCredential -SqlInstance $targetConnection -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" -Identity "SHARED ACCESS SIGNATURE" -SecurePassword (ConvertTo-SecureString $targetSAS -AsPlainText -Force) -Force
$targetCred




StorageAccountName ResourceGroupName PrimaryLocation SkuName      Kind      AccessTier CreationTime          Provisioni
                                                                                                             ngState   
------------------ ----------------- --------------- -------      ----      ---------- ------------          ----------
tempsqlmigstorage  sqlmig            westus2         Standard_LRS StorageV2 Hot        1/14/2020 10:59:39 PM Succeeded 

CloudBlobContainer : Microsoft.Azure.Storage.Blob.CloudBlobContainer
Permission         : Microsoft.Azure.Storage.Blob.BlobContainerPermissions
PublicAccess       : Container
LastModified       : 1/17/2020 7:36:17 PM +00:00
ContinuationToken  : 
Context            : Microsoft.WindowsAzure.Commands.Storage.AzureStorageContext
Name               : backups


ComputerName    : SQLTools2016-3
InstanceName    : MSSQLSERVER
SqlInstance     : SQLTools2016-3
Name            : https://tempsqlmigstorage.blob.core.windows.ne

## Copy Databases to Target Server

The following code will present a list of the databases from the source SQL Server instance.  Select the list of databases to copy and press OK.  The Copy-DbaDatabase CMDLET will take a backup of each database using the azure storage account information above.  Each database backup will then be restored from the blob storage account.  The database restore will use server defaults for database file location and structure.

If the database being restored already exists on the target instance, the *Force* parameter determines the behavior of the notebook:

|Force|Description|
|---|---|
|$true| Overwrite the existing database on the target instance|
|$false| Do not restore the database if it already exists on the target instance|

*Note: This can be a very long running process based on the size of the databases being copied.  The notebook should be allowed to run until the CMDLET completes.*



In [10]:
$databasesToCopy = Get-DbaDatabase -SqlInstance $sourceConnection | Select-Object -Property Name | Out-GridView -PassThru
$databaseList = New-Object System.Collections.ArrayList
foreach ($db in $databasesToCopy)
{
    $databaseList.Add($db.Name)    
}

$copyDatabaseParams = @{
    Database = $databaseList
    Source = $sourceConnection
    Destination = $targetConnection
    BackupRestore = $true
    SharedPath =  "https://$blobStorageAccount.blob.core.windows.net/$containerName"
    Force = $true
    Verbose = $false     
}

Copy-DbaDatabase @copyDatabaseParams

0

Type     Name               Status     Notes
----     ----               ------     -----
Database keep_MigrationTest Successful      


